In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [8]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

file_path = "./train_converted.jsonl"
tokenizer = AutoTokenizer.from_pretrained(
    'allenai/longformer-base-4096'
)
model = AutoModelForQuestionAnswering.from_pretrained(
    'allenai/longformer-base-4096', return_dict=False
)

Some weights of LongformerForQuestionAnswering were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
test = "Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend 'Venite Ad Me Omnes'. Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary."

In [11]:
print(test[514:])

 Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.


In [1]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenized = []
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")
file_path = "./train_converted_enriched.jsonl"
from datasets import Dataset

original_data = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)
        data_line["question"] += 'NIL </ec>'
        original_data.append(data_line)
print(len(original_data))

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


7939


In [14]:
original_data[1]['question'][97:]

'Experience Unlimited album) : album by Experience Unlimited </ec> NIL </ec>'

In [2]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenized = []
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")
file_path = "./aida_train_converted.jsonl"
from datasets import Dataset

original_data_aida = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)
        for training_item in data_line:
            original_data_aida.append(training_item)
print(len(original_data_aida))

18448


In [19]:
original_data[0]

{'context': 'is carts based upon a variety of criteria. IMG also includes a genome annotation pipeline that integrates information from several tools, including  [START_ENT] kegg [END_ENT], Pfam, InterPro, and the Gene Ontology, among others. Users can also type or upload their own gene annotations (called MyIMG gene annotations) ',
 'question': 'Kegg Pipe Organ Builders </ec> KEGG </ec>  NIL </ec>',
 'answers': {'answer_start': [31], 'text': ['KEGG']}}

In [15]:
import random

concat = [] + original_data
random.shuffle(concat)
print(len(concat))
train_original = concat[: int(len(concat) * 0.7)]
eval_original = concat[int(len(concat) * 0.7) :]
dataset = Dataset.from_dict(
    {
        "context": [item["question"] for item in train_original],
        "question": [item["context"] for item in train_original],
        "answers": [item["answers"] for item in train_original],
    }
)
dataset_ev = Dataset.from_dict(
    {
        "context": [item["question"] for item in eval_original],
        "question": [item["context"] for item in eval_original],
        "answers": [item["answers"] for item in eval_original],
    }
)

7939


In [6]:
dataset_ev

Dataset({
    features: ['context', 'question', 'answers'],
    num_rows: 7917
})

In [6]:
dataset[0]

{'context': 'Silent film </ec> Photoplay </ec> Photoplay edition </ec> Photoplay (album) </ec> Film </ec>  NIL </ec>',
 'question': ' film Here We Go Round the Mulberry Bush (1968) where he was cast as Jamie McGregor, a teenager who finds it difficult to lose his virginity.  [START_ENT] photoplay [END_ENT] magazine called Evans a bright and exciting new actor , and The Sunday Telegraph described his screen debut as brilliant .Here We ',
 'answers': {'answer_start': [18], 'text': ['Photoplay']}}

In [16]:
def get_correct_alignement(context, answer):
    """Some original examples in SQuAD have indices wrong by 1 or 2 character. We test and fix this here."""
    gold_text = answer["text"][0]
    start_idx = context.find(gold_text)
    end_idx = start_idx + len(gold_text)
    return start_idx, end_idx
    # if context[start_idx:end_idx] == gold_text:
    #     return start_idx, end_idx       # When the gold label position is good
    # elif context[start_idx-1:end_idx-1] == gold_text:
    #     return start_idx-1, end_idx-1   # When the gold label is off by one character
    # elif context[start_idx-2:end_idx-2] == gold_text:
    #     return start_idx-2, end_idx-2   # When the gold label is off by two character
    # else:
    #     raise ValueError()


# Tokenize our training dataset
def convert_to_features(example):
    try:
        # Tokenize contexts and questions (as pairs of inputs)
        input_pairs = [example["question"], example["context"]]
        encodings = tokenizer.encode_plus(
            input_pairs, pad_to_max_length=True, max_length=512
        )
        context_encodings = tokenizer.encode_plus(example["context"])

        # Compute start and end tokens for labels using Transformers's fast tokenizers alignement methodes.
        # this will give us the position of answer span in the context text

        if example["answers"]["answer_start"][0] == -1:
            print("l'e' che")
            start_idx = example["context"].find("NIL")
            example["answers"]["answer_start"][0] = start_idx
            example["answers"]["text"][0] = "NIL"

        start_idx, end_idx = get_correct_alignement(
            example["context"], example["answers"]
        )
        start_positions_context = context_encodings.char_to_token(start_idx)
        end_positions_context = context_encodings.char_to_token(end_idx - 1)

        # here we will compute the start and end position of the answer in the whole example
        # as the example is encoded like this <s> question</s></s> context</s>
        # and we know the postion of the answer in the context
        # we can just find out the index of the sep token and then add that to position + 1 (+1 because there are two sep tokens)
        # this will give us the position of the answer span in whole example
        sep_idx = encodings["input_ids"].index(tokenizer.sep_token_id)
        start_positions = start_positions_context + sep_idx + 1
        end_positions = end_positions_context + sep_idx + 1

        encodings.update(
            {
                "start_positions": start_positions,
                "end_positions": end_positions,
                "attention_mask": encodings["attention_mask"],
            }
        )
        return encodings
    except Exception as e:
        print("errors", e)

In [17]:
train_ds = dataset.map(convert_to_features)
eval_ds = dataset_ev.map(convert_to_features)

Map:   0%|          | 0/5557 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/rub/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Map:   0%|          | 0/2382 [00:00<?, ? examples/s]

In [15]:
dataset[12]

{'context': 'Dookie United Football Club Dookie Dookie, Victoria Dookie (dog) NIL',
 'question': 'ash; 8 September 1991) was an Australian politician.He was born in Corowa to labourer David Drysdale Morton and Annie Margaret Ellis. He grew up in  [START_ENT] dookie [END_ENT] and served with the 6th Light Horse in World War I. On his return he was a local government officer. On 24 June 1922 he married Winifred Keir',
 'answers': {'answer_start': [35], 'text': ['Dookie, Victoria']}}

In [18]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator

model = AutoModelForQuestionAnswering.from_pretrained(
    "./TrainedModelSpecialTokenNILEL"
).to("cuda")

In [19]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator

data_collator = default_data_collator
# args = TrainingArguments(
#     f"longformer-finetuned-squad",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )

args = TrainingArguments(
    "bert-finetuned-squad",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_eval_batch_size = 8,
    per_device_train_batch_size = 8,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
)

In [20]:
trainer.train()

/home/rub/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: agazzi-ruben99. Use `wandb login --relogin` to force relogin


  0%|          | 0/2085 [00:00<?, ?it/s]

You're using a LongformerTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.3687, 'learning_rate': 2.2863309352517985e-05, 'epoch': 0.72}


  0%|          | 0/298 [00:00<?, ?it/s]

...

  0%|          | 0/298 [00:00<?, ?it/s]

{'eval_loss': 0.29688161611557007, 'eval_runtime': 58.6467, 'eval_samples_per_second': 40.616, 'eval_steps_per_second': 5.081, 'epoch': 2.0}
{'loss': 0.1284, 'learning_rate': 8.47482014388489e-06, 'epoch': 2.16}
{'loss': 0.0643, 'learning_rate': 1.2805755395683455e-06, 'epoch': 2.88}


  0%|          | 0/298 [00:00<?, ?it/s]

{'eval_loss': 0.30743542313575745, 'eval_runtime': 58.0988, 'eval_samples_per_second': 40.999, 'eval_steps_per_second': 5.129, 'epoch': 3.0}
{'train_runtime': 1940.6356, 'train_samples_per_second': 8.59, 'train_steps_per_second': 1.074, 'train_loss': 0.19021372600710934, 'epoch': 3.0}


TrainOutput(global_step=2085, training_loss=0.19021372600710934, metrics={'train_runtime': 1940.6356, 'train_samples_per_second': 8.59, 'train_steps_per_second': 1.074, 'train_loss': 0.19021372600710934, 'epoch': 3.0})

In [21]:
trainer.save_model('./TrainedModelSpecialTokenNILEL_enriched')

In [11]:
import subprocess

# Shutdown
subprocess.run(["shutdown", "-h"])

Shutdown scheduled for Mon 2023-11-13 17:33:04 CET, use 'shutdown -c' to cancel.


CompletedProcess(args=['shutdown', '-h'], returncode=0)

In [ ]:
torch.cuda.empty_cache()

In [59]:
dataset[0]

{'context': 'Give It All (Train song) </ec> Give It All (film) </ec> Give It All </ec>  NIL </ec>',
 'question': "t==Helen McNichol (Tania Raymonde), a high school senior, is in love with Stanford Prescott (Ryan Merriman), her jock boyfriend, and has decided to  [START_ENT] give it all [END_ENT] up to him. At least until she learns that her name is written in the high school football team's secret Bang Book and it is Stanford's j",
 'answers': {'answer_start': [75], 'text': ['NIL']}}

## Prediction and evaluation

In [82]:
import torch
from transformers import LongformerTokenizer, LongformerForQuestionAnswering

index = i
question = dataset_ev[index]["context"]
text = dataset_ev[index]["question"]
print(dataset_ev[index])
input_pairs = [text, question]
encodings = tokenizer.encode_plus(input_pairs, return_tensors="pt").to("cuda")
encodings_context = tokenizer.encode_plus(question, return_tensors="pt").to("cuda")

{'context': 'Air War (game) </ec> Air War </ec> Indo-Pakistani Air War of 1965 </ec> World War II </ec> Aerial warfare </ec>  NIL </ec>', 'question': 'Units, 16) (2004). Osprey Publishing (UK) (). 128 pgs. * Winter, Denis. First of the Few. London: Allen Lane/Penguin, 1982. Coverage of the British  [START_ENT] air war [END_ENT], with extensive bibliographical notes. * Wise, S.F., Canadian Airmen and the First World War: The Official History of the Royal Canadian Air', 'answers': {'answer_start': [113], 'text': ['NIL']}}


In [80]:
for i in range(len(dataset_ev)):
    if dataset_ev[i]["answers"]["text"][0] == 'NIL':
        print(dataset_ev[i])
        break

{'context': 'Air War (game) </ec> Air War </ec> Indo-Pakistani Air War of 1965 </ec> World War II </ec> Aerial warfare </ec>  NIL </ec>', 'question': 'Units, 16) (2004). Osprey Publishing (UK) (). 128 pgs. * Winter, Denis. First of the Few. London: Allen Lane/Penguin, 1982. Coverage of the British  [START_ENT] air war [END_ENT], with extensive bibliographical notes. * Wise, S.F., Canadian Airmen and the First World War: The Official History of the Royal Canadian Air', 'answers': {'answer_start': [113], 'text': ['NIL']}}


In [81]:
dataset_ev[i]

{'context': 'Air War (game) </ec> Air War </ec> Indo-Pakistani Air War of 1965 </ec> World War II </ec> Aerial warfare </ec>  NIL </ec>',
 'question': 'Units, 16) (2004). Osprey Publishing (UK) (). 128 pgs. * Winter, Denis. First of the Few. London: Allen Lane/Penguin, 1982. Coverage of the British  [START_ENT] air war [END_ENT], with extensive bibliographical notes. * Wise, S.F., Canadian Airmen and the First World War: The Official History of the Royal Canadian Air',
 'answers': {'answer_start': [113], 'text': ['NIL']}}

In [83]:
input_ids = encodings["input_ids"]
attention_mask = encodings["attention_mask"]
start_scores, end_scores = model(
    encodings["input_ids"], attention_mask=encodings["attention_mask"]
)

all_tokens = tokenizer.convert_ids_to_tokens(encodings_context["input_ids"][0].tolist())
start = torch.argmax(
    start_scores
)  # Get the most likely beginning of answer with the argmax of the score
end = (
    torch.argmax(end_scores) + 1
)  # Get the most likely end of answer with the argmax of the score


answer_tokens = input_ids[0, start.item() : end.item() + 1]
answer = tokenizer.decode(answer_tokens)



In [25]:
def process_answer(answer):
    modified_answer = answer.replace("</ec>", "")
    modified_answer2 = modified_answer.replace("</", "")
    modified_answer2 = modified_answer2.split(':')[0]
    return modified_answer2.strip()

In [85]:
print(process_answer(answer))

NIL


In [26]:
import pywikibot


def get_wikidata(wikipedia_title):
    try:
        site = pywikibot.Site("en", "wikipedia")  # Connect to the English Wikipedia
        page = pywikibot.Page(site, wikipedia_title)  # Get the Wikipedia page
        item = pywikibot.ItemPage.fromPage(page)  # Get the associated Wikidata item
        item.get()
        if "en" in item.descriptions:  # Fetch all data of the Wikidata item
            return item.descriptions["en"]  # Return the English description
        else:
            return ""
    except:
        return ""

In [2]:
def make_prediction(data_entry):
    question = data_entry["input"]
    context = ""
    for item in data_entry["candidates"]:
        desc = get_wikidata(item)
        context += item + f" : {desc} </ec> "
    input_pairs = [question, context]

    encodings = tokenizer.encode_plus(input_pairs, return_tensors="pt").to("cuda")
    start_scores, end_scores = model(
        encodings["input_ids"], attention_mask=encodings["attention_mask"]
    )

    start = torch.argmax(
        start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    end = (
        torch.argmax(end_scores) + 1
    )  # Get the most likely end of answer with the argmax of the score

    answer_tokens = encodings["input_ids"][0, start.item() : end.item() + 1]
    answer = process_answer(tokenizer.decode(answer_tokens))
    return {"correct": data_entry["output"], "predicted": answer}

In [3]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

model = AutoModelForQuestionAnswering.from_pretrained(
    "./TrainedModelSpecialTokenNILEL"
).to("cuda")

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
from tqdm import tqdm
import json
import torch

results = []
file_path = "./train.jsonl"
dataset = []
with open(file_path, "r") as file:
    for line in file:
        # Load each line as a JSON object
        data_line = json.loads(line)
        for item in data_line:
            dataset.append(item)

for item in tqdm(dataset):
    pred = make_prediction(item)
    results.append(pred)

  0%|          | 0/7939 [00:00<?, ?it/s]


NameError: name 'get_wikidata' is not defined

In [41]:
results[0]

{'correct': [{'answer': 'Tasmanian Legislative Council',
   'provenance': [{'title': 'Tasmanian Legislative Council'}]}],
 'predicted': 'Tasmanian Legislative Council'}

In [7]:
correct = 0
for result in tqdm(results):
  if result['predicted'] == result['correct'][0]['answer']:
    correct += 1

100%|██████████| 6821/6821 [00:00<00:00, 1016606.77it/s]


In [10]:
print(f"accuracy of the model is {correct/len(results)}")

accuracy of the model is 0.7312710746224894
